# Uninformed Tree Search Without Duplicate Detection
In this lab you are going to implement basic tree-search methods (without duplicate states detection) - BFS (Breadth First Search), DFS (Depth First Search), DFS-L (Depth First Search with Limited Depth), DFID (Depth First Iterative Deepening).

Two widespread domains will be considered - 15-puzzle and Panckakes. For 15-puzzle the code that defines the state and the getSuccesors function is already available. For Panckakes - you have to code it yourself. All search methods have to be coded by you as well, using the code stubs provided.

Run every cell of the notebook and complete the described tasks. Good luck!

In [1]:
import copy

from collections import deque, defaultdict

# Gem Puzzle (15-puzzle or n-puzzle)
The 15-puzzle (also called Gem Puzzle, Boss Puzzle, Game of Fifteen, Mystic Square and many others) is a sliding puzzle that consists of a frame of numbered square tiles in random order with one tile missing (see picture below). The puzzle also exists in other sizes, particularly the smaller 8-puzzle. If the size is 3×3 tiles, the puzzle is called the 8-puzzle or 9-puzzle, and if 4×4 tiles, the puzzle is called the 15-puzzle or 16-puzzle named, respectively, for the number of tiles and the number of spaces. The object of the puzzle is to place the tiles in order by making sliding moves that use the empty space. Note, that half of the starting positions for the n-puzzle are impossible to resolve, no matter how many moves are made. [[Wikipedia](https://en.wikipedia.org/wiki/15_puzzle)]. 

![puzzle](Image/GemPuzzle.png)

## Representation of a search state for the Gem Puzzle

Implementing SearchState (or simply - State) in code is a very important first step needed to tackle any search problem.

The suggested implementation (not the only one possible, obviously) of the GemPuzzleState is comprised of the following fields: 
- size - width of game field 
- tileList - tile positions represented as a list of integers. This list is expected to contain values from 1 to *size* * *size*. Each integer value corresponds to a tile and the position in the list (index) corresponds to the position of the tile on the game field. Tile with the value *size* * *size* is assumed to represent blank position.
- parent - pointer to the parent-state. Parent is a predecessor of the state in the search-tree. It is used to reconstrruct a path to that state from the start state (root of the search tree).
- blankPos - position of empty tile in tileList. Explicitly storing the position of a blank helps to generate successors faster.



In [2]:
class GemPuzzleState:

    # Constructor. Sets tile positions + some basic checks.
    def __init__(self, tileList):
        self.tileList = tileList
        self.size = int(len(tileList) ** 0.5)
        blankValue = self.size ** 2
        if (blankValue != len(tileList)):
            raise Exception("The tile list must contain the number of elements which is equal to the square of an integer!")

        # Memorizing the position of a blank tile
        # Technically, there is no need to do so, but it makes to get the successors a bit faster
        self.blankPos = -1 
        for i in enumerate(tileList):
            if (i[1] == blankValue):
                self.blankPos = i[0]

                
        if (self.blankPos == -1):
            raise Exception("State should contains max value as position to blank tile")     
        
        # The parent state (predecessor in the search tree) will be set up by the search algorithm.
        self.parent = None
       
    # Comparing one state with the other state. This is needed e.g. to test whether we reached the goal state.
    def __eq__(self, other):
        for i in enumerate(self.tileList):
            if i[1] != other.tileList[i[0]]:
                return False
        return True


    # Printing the state as tile matrix
    def Print(self):
        res = []
        charTileList = list(map(str, self.tileList))
        charTileList[self.blankPos] = '_'
        for rowStart in range(0, len(charTileList), self.size):
            res.append(charTileList[rowStart:rowStart+self.size])
        print()
        print('\n'.join([''.join(['{:2}'.format(item) for item in row]) 
            for row in res]))


### Get Succesors
Implementing GetSuccessors function is another important step to tackle any search problem.

This function is presumed to take a particular search state as the input and to return all possible succesors states. i.e. the ones that result from applying all applicable actions to the input state. 

In case of GemPuzzle the succesors correspond to the board states resulting from moving blank up/down/left/right. If blank goes out of the field after a move, such successor should obviously be discarded.

In [25]:
def GetSuccessors(state):
    delta = [[0, 1], [1, 0], [0, -1], [-1, 0]]
    successors = []
    for d in delta:
        row = state.blankPos // state.size #identifying the row in which blank is located
        col = state.blankPos % state.size #identifying the column in which blank is located
        row += d[0] #computing new row for blank (corresponding to a particular move encoded via 'd')
        col += d[1] #computing new column for blank (corresponding to a particular move encoded via 'd')
        
        #if the new position of a blank is valid (i.e. it is still within the field) then
        #a corresponding sucessor should be added to the succesors' list
        if (0 <= row < state.size) and (0 <= col < state.size):
            newState = copy.deepcopy(state)
            newState.visited = False
            newBlankPos = row * state.size + col
            newState.tileList[newState.blankPos] = newState.tileList[newBlankPos] #moving tile
            newState.tileList[newBlankPos] = newState.size ** 2 #setting blank
            newState.blankPos = newBlankPos
            newState.parent = state
            successors.append(newState)

    return successors

### Goal check

Handy function that returns `True` if the input `state` corresponds to the goal one (i.e. all tiles are in their places), and `False` otherwise

In [4]:
def StateIsGoal(state):
    for i in range(1, len(state.tileList)):
        if(state.tileList[i-1] > state.tileList[i]):
            return False
    return True

### Path checking
Auxiliary function that takes the `lastState` and checks whether this state is a goal. If yes, it unwinds the path using the backpointers and checks whether each successor is reachable from its predecessor.

In [5]:
def CheckPath(lastState):
        curr = lastState
        if not StateIsGoal(curr):
            print("Goal was not reached!")
            return False

        while(curr.parent is not None):
            prev = curr.parent
            if (curr not in GetSuccessors(prev)):
                print("Unacceptable step!")
                return False                
            curr = prev
        return True

### Path unwinding
Typically the paths are not stored within a search explicitly, but rather implicitly via the parent pointers (pointing to the predecessor in the search tree). Thus, when we reach the goal state and want to reconstruct the whole path we need to trace the parent pointers back to the root of the tree.

Technically this function takes a state `lastState` as an input and returns a path to this state from the root of the tree.

In [6]:
def GetPath(lastState):
    path = []
    curr = lastState
    while curr is not None:
        path.append(curr)
        curr = curr.parent
    return path


## Automated tests to check the implementations of the search algorithms
When you finish implementing search algorithms you need to test them, right? The following functions will help you in that. They take your search algorithm as an input and run it on a single simple test (`SimpleTest`) and on a series of a more involved test (`MassiveTest`).

These automated tests assume that the seach function, passed as the input, has the following structure:

`SearchFunction(startState, *optional arguments*) -> (pathFound, lastState)`, where

- startState -- initial state
- *optional arguments* -- additional parameters of the search function (if needed), passed via `*args`
- pathFound -- result of the search, `True` if path was found, `False` otherwise
- lastState -- last state of path. `None` if path was not found

SimpleTest runs `SearchFunction` on a simple 2x2 sliding puzzle instance (encoded as [3, 1, 2, 4]).

The output is as follows:
- 'Path is OK' and list of states of path -- path was found and it is correct
- 'Path is not OK(' -- path was found but it is not correct 
- 'Path not found(' -- path was not found 
- 'Execution error' -- an error occurred while executing the `SearchFunction` or path validation function

In [7]:
def SimpleTest(SearchFunction, *args):
    startState = GemPuzzleState([3,1,2,4])
    try:
        result = SearchFunction(startState, *args)
        curr = result[1]
        if(result[0]):
            if(CheckPath(curr)):
                print("Path is OK!")
                path = GetPath(result[1])

                while len(path) != 0:
                    s = path.pop()
                    s.Print()
            else:
                print("Path is not OK(")
        else:
            print("Path not found(")
    except Exception as e:
        print("Execution error")
        print(e)

MassiveTest runs `SearchFunction` on set of different tasks (stored in `Data/taskGem.txt`). Initially this file contains 4 different 2x2 sliding puzzle instances and 4 different 3x3 sliding puzzle instances (you can add more if you want).

The output is similar to `SimpleTest` (however the full paths for solved instances are not displayed):

- 'Path is OK' -- path was found and it is correct
- 'Path os not OK(' -- path was found but it is not correct 
- 'Path not found(' -- path was not found
- 'Execution error' -- an error occurred  while executing the `SearchFunction` or path validation function
 

In [8]:
def MassiveTest(SearchFunction, *args):
    tasksFile = open('Data/tasksGem.txt')
    count = 0
    for l in tasksFile:
        count += 1
        state = list(map(int, l.split()))
        task = GemPuzzleState(state)
        try:
            result = SearchFunction(task, *args)
            curr = result[1]
            if(result[0]):
                if(CheckPath(curr)):
                    print(count, "Path is OK!")
                else:
                    print(count, "Path is not OK(")
            else:
                print(count, "Path not found(")
        except Exception as e:
            print(count, "Execution error")
            print(e)
        

## Breadth-First Search (BFS)

In [9]:
# TODO Implementation of Breadth-First Search algorithm

def BFS(start):
    pathFound = False
    resState = None
    
    #CODE HERE
    queue = deque([start])
    while queue:
        state = queue.popleft()
        if StateIsGoal(state):
            return (True, state)
        for successor in GetSuccessors(state):
            successor.parent = state
            queue.append(successor)
    
    return (pathFound, resState)


In [10]:
# Test your BFS on simple task
SimpleTest(BFS)

Path is OK!

3 1 
2 _ 

3 1 
_ 2 

_ 1 
3 2 

1 _ 
3 2 

1 2 
3 _ 


In [11]:
# If simple test is OK, you should check your implementation in massive test. The rest of the search algorithms are checked in the same way.
MassiveTest(BFS)

1 Path is OK!
2 Path is OK!
3 Path is OK!
4 Path is OK!
5 Path is OK!
6 Path is OK!
7 Path is OK!
8 Path is OK!


## Depth-First Search (DFS)

In [58]:
# TODO Implementation of Depth-First Search algorithm

def DFS(start):

    #CODE HERE
    stack = []
    stack.append(start)
    
    while stack:
        state = stack.pop()
        
        if StateIsGoal(state):
            return (True, state)
        
        for successor in reversed(GetSuccessors(state)):
            successor.parent = state
            stack.append(successor)
            
    return (False, None)

# def DFS(state):

#     pathFound = False
#     resState = None
    
#     if StateIsGoal(state):
#         pathFound = True
#         resState = state
#         return (pathFound, resState)
    
#     successors = GetSuccessors(state)
#     for s in successors:
#         s.parent = state
#         pathFound, resState = DFS(s)
#         if pathFound:
#             break
        
#     return (pathFound, resState)


Using DFS, you will most likely encounter the fact that this algorithm overcomes the threshold of recursive calls, after which the execution will interrupted. 

In [37]:
SimpleTest(DFS)

KeyboardInterrupt: 

But you can create such a simple task, which can be solved by DFS

In [33]:
def DFSSimpleTest(SearchFunction, *args):
    # Insert the task that can be solved by a DFS solver
    YourTileList = [1, 4, 3, 2]
    
    startState = GemPuzzleState(YourTileList)
    try:
        result = SearchFunction(startState, *args)
        curr = result[1]
        if(result[0]):
            if(CheckPath(curr)):
                print("Path is OK!")
                path = GetPath(result[1])

                while len(path) != 0:
                    s = path.pop()
                    s.Print()
            else:
                print("Path is not OK(")
        else:
            print("Path not found(")
    except Exception as e:
        print("Execution error")
        print(e)

In [59]:
DFSSimpleTest(DFS)

Path is OK!

1 _ 
3 2 

1 2 
3 _ 


In [ ]:
# There is no need to start MassiveTest
#MassiveTest(DFS)

## Depth First Search with Limited Depth
One of the way to solve problem of overcoming the threshold of recursive calls is explicitly limit the to depth of the search tree by passing an appropriate parameter `limit` to the search algorithm. The second parameter `depth` is a technical one needed for the implementation. It represents the current depth of the search. Initially (when invoked on the start state of the problem) it is, indeed, equal to 0.

In [67]:
# TODO Implementation of Depth-Limited Search algorithm

# def DFSL(start, limit):

#     #CODE HERE
#     stack = []
#     depth = 0
#     stack.append(start)
    
#     while stack and depth < limit:
#         depth += 1
#         state = stack.pop()
        
#         if StateIsGoal(state):
#             return (True, state)
        
#         for successor in reversed(GetSuccessors(state)):
#             successor.parent = state
#             stack.append(successor)
                
#     return (False, None)

def DFSL(state, limit, depth):
    
    pathFound = False
    resState = None
    
    if StateIsGoal(state):
        return (True, state)
    
    if depth < limit:
        for successor in GetSuccessors(state):
            successor.parent = state
            pathFound, resState = DFSL(successor, limit, depth + 1)
            if pathFound:
                return (pathFound, resState)

    return (pathFound, resState)

Let's check this approach with several different limits

In [68]:
SimpleTest(DFSL, 3, 0)

Path not found(


In [69]:
SimpleTest(DFSL, 5, 0)

Path is OK!

3 1 
2 _ 

3 1 
_ 2 

_ 1 
3 2 

1 _ 
3 2 

1 2 
3 _ 


In [70]:
SimpleTest(DFSL, 10, 0)

Path is OK!

3 1 
2 _ 

3 1 
_ 2 

3 1 
2 _ 

3 1 
_ 2 

3 1 
2 _ 

3 1 
_ 2 

3 1 
2 _ 

3 1 
_ 2 

_ 1 
3 2 

1 _ 
3 2 

1 2 
3 _ 


In [71]:
MassiveTest(DFSL, 3, 0)

1 Path not found(
2 Path not found(
3 Path is OK!
4 Path is OK!
5 Path is OK!
6 Path not found(
7 Path not found(
8 Path not found(


In [72]:
MassiveTest(DFSL, 5, 0)

1 Path is OK!
2 Path is OK!
3 Path is OK!
4 Path is OK!
5 Path is OK!
6 Path is OK!
7 Path is OK!
8 Path not found(


In [73]:
MassiveTest(DFSL, 10, 0)

1 Path is OK!
2 Path is OK!
3 Path is OK!
4 Path is OK!
5 Path is OK!
6 Path is OK!
7 Path is OK!
8 Path is OK!


## Depth First Iterative Deepening Search (DFID)
Finally let's sequentially invoke DFS with increasing depth limits. This is called the Depth First Iterative Deepening algorithm. It will inded find a solution if one exists.

In [75]:
# TODO Implementation of Iterative-Deepening Depth-First Search
def DFID(state):
    iteration = 1
    pathFound, resState = DFSL(state, iteration, 0)
    
    while not pathFound:
        iteration += 1
        pathFound, resState = DFSL(state, iteration, 0)
        
    return (pathFound, resState)
    
    #replace return (False, None) with an appropriate code

In [76]:
SimpleTest(DFID)

Path is OK!

3 1 
2 _ 

3 1 
_ 2 

_ 1 
3 2 

1 _ 
3 2 

1 2 
3 _ 


In [77]:
MassiveTest(DFID)

1 Path is OK!
2 Path is OK!
3 Path is OK!
4 Path is OK!
5 Path is OK!
6 Path is OK!
7 Path is OK!
8 Path is OK!


## Pancake Sorting

![Example](Image/Cat.jpg)

Pancake sorting is the colloquial term for the mathematical problem of sorting a disordered stack of pancakes in order of size when a spatula can be inserted at any point in the stack and used to flip all pancakes above it (See picture below) [[Wikipedia](https://en.wikipedia.org/wiki/Pancake_sorting)].

![Example](Image/Pancake.png)

### Representation of a state
In this task you should create your own implementation of pancake sorting problem state (and all related funtions) with your own test data. Note, that the interface of the state-class must be the same as for the `GemPuzzleState` thus all the machinery introduced before (e.g. automated tests) will work out-of-the-box. 

In [ ]:
class PancakeDish:
    # Constructor
    def __init__(self, pancakes):
        # TODO
       
    # Compare with other state. Using for path checking
    def __eq__(self, other):
        # TODO
        # return True


    # Prints state in form of tile matrix
    def Print(self):
        # TODO



In [ ]:
def GetSuccessors(state):
    # TODO
    # successors = []

    # return successors

In [ ]:
def StateIsGoal(state):
    # TODO


In [ ]:
def SimpleTest(SearchFunction, *args):
    startState = PancakeDish('''Insert your simple task here''')
    try:
        result = SearchFunction(startState, *args)
        curr = result[1]
        if(result[0]):
            if(CheckPath(curr)):
                print("Path is OK!")

                stack = []
                curr = result[1]
                while curr is not None:
                    stack.append(curr)
                    curr = curr.parent

                while len(stack) != 0:
                    s = stack.pop()
                    s.Print()
            else:
                print("Path is not OK(")
        else:
            print("Path not found(")
    except Exception as e:
        print("Execution error")
        print(e)

In [ ]:
def MassiveTest(SearchFunction, *args):
    # TODO your file with at least 8 different tasks
    asksFile = open('Data/YourTaskFile.txt')
    count = 0
    for l in tasksFile:
        count += 1
        state = list(map(int, l.split()))
        task = PancakeDish(state)
        try:
            result = SearchFunction(task, *args)
            curr = result[1]
            if(result[0]):
                if(CheckPath(curr)):
                    print(count, "Path is OK!")
                else:
                    print(count, "Path is not OK(")
            else:
                print(count, "Path not found(")
        except Exception as e:
            print(count, "Execution error")
            print(e)

## Lets check!

In [ ]:
SimpleTest(BFS)

In [ ]:
MassiveTest(BFS)

In [ ]:
SimpleTest(DFS)

In [ ]:
#MassiveTest(DFS)

In [ ]:
SimpleTest(DLS, 2, 0)
SimpleTest(DLS, 5, 0)
SimpleTest(DLS, 10, 0)


In [ ]:
MassiveTest(DLS, 2, 0)

In [ ]:
MassiveTest(DLS, 5, 0)

In [ ]:
MassiveTest(DLS, 10, 0)

In [ ]:
SimpleTest(DFID)

In [ ]:
MassiveTest(DFID)

## References
1. https://cs.lmu.edu/~ray/notes/usearch/
2. https://lanwuwei.github.io/courses/SP19/3521_slides/4-Informed-Search_1.pdf